# Import Libraries and Setup Parameters and Settings
## Import the pandas package

In [1]:
import pandas as pd

## Adjust Settings for optimized viewing

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)

# Import Data from CSV File
We are working with a csv file called 'orders_complete'. This file contains a concatenation of each order by items sold. As a result, this imported dataframe will have many null values as not each row represents a unique order but a specific item from a unique. Furthermore, only some customer and location information is provided, often when the orders were done online.

In [3]:
df_raw = pd.read_csv('Jupyter Notebook Projects/Shopify/orders_complete.csv')

# Data Clean Up
Before we do any calculation or analysis, we much first clean our data. We will use the original dataframe initially as our location for all changes. Once we have enough to work with, we will select particular columns from this dataframe and call it our working dataframe: df_core. This method is useful as a project continue, you can simply revisit the cleaning portion of the code to adjust whatever is needed, and just add this to the working dataframe.

## 1. ID Name for Orders
This data can be thought of as a concatenation of every order's line items. For example, if customer 1 orders a shirt and a CD, while customer 2 orders a shirt, then we will see 3 rows: Each row corresponding to each line item. Each order has a unique Order ID, initially in column called 'Name', and has the format '#1234'. Let's change the name of the column and remove the '#' from all the values:

###  - rename 'Name' to 'OrderID'

In [4]:
df_raw = df_raw.rename(columns={'Name':'OrderID'})

###  - Then remove the '#'

In [5]:
df_raw['OrderID'] = df_raw['OrderID'].str.replace('#','')

## 2. Filling NaN's for Matching OrderIDs
We mentioned that df_raw is essentially a concatentation of multiple order's line items. For each case where multiple items are part of an order (that is associated with a match OrderID), fields such as 'Billing Name' and 'Billing Street' are only filled in for the first item. This section works to fill null values present with the proper matching details. Additionally, we reorder the columns.

In [6]:
df = df_raw.groupby('OrderID').fillna(method='ffill')
df['OrderID'] = df_raw['OrderID']
df_raw = df.iloc[:, [-1] + list(range(df.shape[1]-1))].copy()

## 3. Number of Items Per Order

In [7]:
df_raw['Items'] = df_raw.groupby('OrderID')['Lineitem quantity'].transform('sum') # count total items per order

## 4. Create a Year and Month Column
There are a few entries in this dataframe that can provide date information, we will consider the 'Created at' column. We can keep it simple and just extract the respective section of the string to get the Month and Year:

In [8]:
df_raw['Month'] = df_raw['Created at'].str[5:7]
df_raw['Year'] = df_raw['Created at'].str[0:4]
df_raw['Datetime'] = df_raw['Created at'].str.slice(stop=-6)
df_raw['UTC'] = df_raw['Created at'].str.slice(start=-6)
df_raw['Datetime'] = pd.to_datetime(df_raw['Datetime'], format='%Y-%m-%d %H:%M:%S')
# df_raw['Datetime'].dt.year
df_raw['Daydt'] = df_raw['Datetime'].dt.day
df_raw['Monthdt'] = df_raw['Datetime'].dt.month
df_raw['Yeardt'] = df_raw['Datetime'].dt.year

In [9]:
false_rows = df_raw[df_raw['Datetime'].isna()]
false_rows[['OrderID','Financial Status','Created at','Datetime']]

,OrderID,Financial Status,Created at,Datetime


## 5. Adding Item Identifiers
Often analysis is based upon if an item is of a certain type. This could be simply a shirt versus a CD, or perhaps part of a unit sale or event. When the item has a condition that can be searched, we can do this here and add a column indicating as such. 

### 5.1 Online Orders
All items that are part of an online orders will be given a Boolean value 'True'. If our point of sale has a non-NaN. (NOTE: This technically could have been done prior to the Filling NaN's step, as this identifier is associated with the Order, not the item.)

In [10]:
df_raw['Online'] = True  # Define initial 'Online' column as True
df_raw.loc[df_raw['Employee'].notnull(), 'Online'] = False

### 5.2 Grand Ol' Productions 

In [11]:
df_raw['GOPC_Item'] = False
df_raw.loc[df_raw['Lineitem name'].str.contains('Grand', case=True), "GOPC_Item"] = True
df_raw.loc[df_raw['Lineitem name'].str.contains('GOCS', case=True), "GOPC_Item"] = True

### 5.3 Pledge Campaign

In [12]:
df_raw['Pledge_Item'] = False  # Define initial 'Pledge' column as False
df_raw.loc[df_raw['Lineitem name'].str.contains('Pledge', case=False), "Pledge_Item"] = True  # look if Item_Notes has 'Pledge'

### 5.4 Free Shipping and Handling Offer

In [13]:
df_raw['FreeSH_Item'] = False  # Define initial 'FreeSH' column as False
df_raw.loc[df_raw['Lineitem name'].str.contains('FREE', case=True), "FreeSH_Item"] = True  # look if Item_Notes contains 'FREE'
df_raw.loc[~(df_raw['Datetime'] >= pd.to_datetime('2022-03-01')), 'FreeSH_Item'] = False

# df_raw.loc[df_raw['FreeSH'] == True, "FreeSH_Date"] = df_raw.loc[df_raw['FreeSH'] == True, 'Created at']

In [14]:
df_raw[df_raw['OrderID']=='1503']

,OrderID,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Items,Month,Year,Datetime,UTC,Daydt,Monthdt,Yeardt,Online,GOPC_Item,Pledge_Item,FreeSH_Item
3941,1503,cliffordgibbs@hotmail.co.uk,paid,2021-06-17 14:01:14 -0500,fulfilled,2021-06-28 18:03:36 -0500,no,USD,45.0,15.44,0.0,60.44,NaN,0.0,First Class Package International,2021-06-17 14:01:12 -0500,1,Things We Carry CD,15.0,NaN,NaN,True,True,fulfilled,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 27 Ashton Court,46 Western Road,NaN,Poole,BH13 6EU,ENG,GB,+447742631070,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 27 Ashton Court,46 Western Road,NaN,Poole,BH13 6EU,ENG,GB,+447742631070,NaN,NaN,NaN,Shopify Payments,c21344658391229.1,0.0,Blue Water Highway,0.0,NaN,NaN,NaN,3.907472e+12,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.477426e+11,NaN,NaN,England,England,c21344658391229.1,NaN,NaN,c21344658391229.1,5,06,2021,2021-06-17 14:01:12,-0500,17,6,2021,True,False,False,False
3942,1503,cliffordgibbs@hotmail.co.uk,paid,2021-06-17 14:01:14 -0500,fulfilled,2021-06-28 18:03:36 -0500,no,USD,45.0,15.44,0.0,60.44,NaN,0.0,First Class Package International,2021-06-17 14:01:12 -0500,1,Paper Airplanes CD,15.0,NaN,NaN,True,True,fulfilled,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 27 Ashton Court,46 Western Road,NaN,Poole,BH13 6EU,ENG,GB,+447742631070,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 27 Ashton Court,46 Western Road,NaN,Poole,BH13 6EU,ENG,GB,+447742631070,NaN,NaN,NaN,Shopify Payments,c21344658391229.1,0.0,Blue Water Highway,0.0,NaN,NaN,NaN,3.907472e+12,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.477426e+11,NaN,NaN,England,England,c21344658391229.1,NaN,NaN,c21344658391229.1,5,06,2021,2021-06-17 14:01:12,-0500,17,6,2021,True,False,False,False
3943,1503,cliffordgibbs@hotmail.co.uk,paid,2021-06-17 14:01:14 -0500,fulfilled,2021-06-28 18:03:36 -0500,no,USD,45.0,15.44,0.0,60.44,NaN,0.0,First Class Package International,2021-06-17 14:01:12 -0500,1,Heartbreak City Stripped CD,15.0,NaN,BWH-1000,True,True,fulfilled,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 27 Ashton Court,46 Western Road,NaN,Poole,BH13 6EU,ENG,GB,+447742631070,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 27 Ashton Court,46 Western Road,NaN,Poole,BH13 6EU,ENG,GB,+447742631070,NaN,NaN,NaN,Shopify Payments,c21344658391229.1,0.0,Blue Water Highway,0.0,NaN,NaN,NaN,3.907472e+12,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.477426e+11,NaN,NaN,England,England,c21344658391229.1,NaN,NaN,c21344658391229.1,5,06,2021,2021-06-17 14:01:12,-0500,17,6,2021,True,False,False,False
3944,1503,cliffordgibbs@hotmail.co.uk,paid,2021-06-17 14:01:14 -0500,fulfilled,2021-06-28 18:03:36 -0500,no,USD,45.0,15.44,0.0,60.44,NaN,0.0,First Class Package International,2021-06-17 14:01:12 -0500,2,FREE - Heartbreak City CD,0.0,15.0,BWH-1000,True,False,fulfilled,Clifford Gibbs,"Flat 27 Ashton Court, 46 Western Road",Flat 

# Defining our Working Dataframe
Here we will define the df_core.

Here we will take the opportunity to rename all columns and move them in the position that works for our needs.

In [15]:
df_core = df_raw[['OrderID', 'Monthdt', 'Yeardt', 'Datetime', 'Billing Name', 'Billing City', 'Billing Province',
                  'Billing Country', 'Lineitem name','Lineitem price', 'Lineitem quantity', 'Subtotal', 'Items', 'Online', 'FreeSH_Item', 'GOPC_Item']]
df_core.columns = ['OrderID', 'Month', 'Year', 'Date', 'Name', 'City', 'State',
                   'Country', 'Lineitem', 'Lineitem Price', 'Lineitem Quantity', 'Order_Subtotal', 'Total_Order_Items', 'Online', 'FreeSH_Item', 'GOPC_Item']

# Testing

In [16]:
# df_core['Date'] = pd.to_datetime(df_core['Date'], format='%Y-%m-%d %H:%M:%S %z')
# df_core['Formatted_Date'] = df_core['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# df_core['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# df_core['my_timestamp_column'].dt.strftime('%Y-%m-%d %H:%M:%S')
# df_core['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Insights

## Questions

### 1. How many people took part in the Free Shipping and Handling Offer?

In [20]:
freeSH_orders = df_core[df_core['FreeSH_Item'] == True]
# len(freeSH_orders['Name'].unique())
freeSH_orders.sort_values(by='Date')

,OrderID,Month,Monthdt,Year,Yeardt,Date,Name,City,State,Country,Lineitem,Lineitem Price,Lineitem Quantity,Order_Subtotal,Total_Order_Items,Online,FreeSH_Item,GOPC_Item
2899,2236,03,3,2022,2022,2022-03-03 06:08:22,Jennifer Armstrong,Massillon,OH,US,FREE - Paper Airplanes CD,0.0,1,0.0,1,True,True,False
2898,2237,03,3,2022,2022,2022-03-03 15:01:22,John Moore,Borger,TX,US,FREE - Paper Airplanes CD,0.0,1,0.0,1,True,True,False
2897,2238,03,3,2022,2022,2022-03-03 19:01:40,Debbie Dearth,KANSAS CITY,KS,US,FREE - Paper Airplanes CD,0.0,1,0.0,1,True,True,False
2896,2239,03,3,2022,2022,2022-03-05 19:15:16,Lisa Gomez,Davenport,IA,US,FREE - Paper Airplanes CD,0.0,1,15.0,2,True,True,False
2894,2240,03,3,2022,2022,2022-03-06 12:37:15,Kirsteen Williams,Foxton,ENG,GB,FREE - Paper Airplanes CD,0.0,1,15.0,2,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,4075,12,12,2022,2022,2022-12-04 04:02:51,Jason Laymon,Mount Vernon,OH,US,FREE - Heartbreak City CD,0.0,1,0.0,1,True,True,False
320,4094,12,12,2022,2022,2022-12-05 19:34:40,glenn jacobs,Greenville,SC,US,FREE - Heartbreak City CD,0.0,1,30.0,3,True,True,False
70,4272,01,1,2023,2023,2023-01-03 15:36:09,Paul Cook,Hayesville,NC,US,FREE - Heartbreak City CD,0.0,1,15.0,2,True,True,False
45,4290,01,1,2023,2023,2023-01-06 13:53:57,Norman Lazarus,Elkridge,MD,US,FREE - Heartbreak City CD,0.0,1,0.0,1,True,True,False


### 2. How many bought free Cd and the upsell offer?

In [18]:
freeSH_orders[freeSH_orders['Total_Order_Items']>1]['OrderID'].count()

361

### 3. What City/state do our customers reside in?

### 4. Cities/states with largest amount purchased? This is obviously Texas, but what are the top 5 or 10 other states/markets?

### 5. How many people bought free CD and came back for another purchase at a later date?

In [16]:
len(df_core[df_core['FreeSH_Item'] == True]['OrderID'].unique())

1414